<a href="https://colab.research.google.com/github/samsatp/NLP-contest3/blob/main/exp3-enc_dec_attn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!git clone https://github.com/samsatp/NLP-contest3.git

!gdown 1LGWy5VGVk6TXz_ZaYwGBvwJc5vFBzeZe
!unzip /content/data.zip
!rm /content/data.zip

Cloning into 'NLP-contest3'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 23 (delta 7), reused 23 (delta 7), pack-reused 0
Unpacking objects: 100% (23/23), done.
Downloading...
From: https://drive.google.com/uc?id=1LGWy5VGVk6TXz_ZaYwGBvwJc5vFBzeZe
To: /content/data.zip
100% 24.1M/24.1M [00:00<00:00, 26.4MB/s]
Archive:  /content/data.zip
  inflating: dev_auto_tok.tsv        
  inflating: dev_entities.json       
  inflating: newly_tokenized/dev_auto_tok.tsv  
  inflating: newly_tokenized/dev_entities.json  
  inflating: newly_tokenized/train_auto_tok.tsv  
  inflating: newly_tokenized/train_entities.json  
  inflating: raw/dev_set.txt         
  inflating: raw/test_set.txt        
  inflating: train_auto_tok.tsv      
  inflating: train_entities.json     


In [15]:
!pip install tensorflow-addons==0.11.2
!mkdir /content/result
!mkdir /content/models

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [16]:
import importlib  
preprocess = importlib.import_module("NLP-contest3.util.preprocess")
lst20utils = importlib.import_module("NLP-contest3.lst20utils")

In [17]:
train_seqs = preprocess.get_sequence("/content/train_auto_tok.tsv")
dev_seqs = preprocess.get_sequence("/content/dev_auto_tok.tsv")

X_train_raw, Y_train_raw = ['|'.join([e[0] for e in row]) for row in train_seqs], [[e[1] for e in row] for row in train_seqs]
X_dev_raw, Y_dev_raw = ['|'.join([e[0] for e in row]) for row in dev_seqs], [[e[1] for e in row] for row in dev_seqs]

In [18]:
tags = ['<BOS>', 'I_LOC', 'B_LOC', 'B_MEA', 'B_NUM', 'I_ORG', 'I_DES', 'I_MEA', 'B_TRM', 'B_DTM', 'B_DES', 'B_BRN', 'I_PER', 'I_NUM', 'B_ORG', 'I_DTM', 'I_BRN', 'B_PER', 'I_TRM', 'I_TTL', 'B_TTL']

In [19]:
X_train_raw[0], Y_train_raw[0]

('ธรรมนูญ|แชมป์|สิงห์|คลาสสิก|กวาด|รางวัล|แสน|สี่|หมื่น|บาท',
 ['B_PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])

In [37]:
import tensorflow as tf
import tensorflow_addons as tfa

class SeqData:
    def __init__(self, X, Y):
        self.X = X  # List of texts
        self.tokenizer = tf.keras.preprocessing.text.Tokenizer(
                            filters='',
                            lower=False,
                            split='|',
                            char_level=False,
                            oov_token='<OOV>'
                        )
        self.ne_stringLookup = tf.keras.layers.StringLookup(
                            max_tokens=None,
                            num_oov_indices=1,
                            mask_token='O',
                            oov_token='<OOV>',
                            vocabulary=tags,
                            output_mode='int',
                            sparse=False,
                            pad_to_max_tokens=False
                        )
        self.tokenizer.fit_on_texts(X)
    
    def padding(self, seqs, dtype='int32', value=0.0):
        return tf.keras.preprocessing.sequence.pad_sequences(seqs, padding='post', dtype=dtype, value=value)

    def preprocess_ne_seq(self, seq):
        seq.insert(0, '<BOS>')
        return seq

    def get_bool_mask(self, seqs):
        mask = tf.ragged.constant(seqs)
        return tf.ones_like(mask, dtype='int32').to_tensor(default_value=0)

    def load_dataset(self, X, Y, BATCH_SIZE=64):

        # create boolean mask
        Y = [self.preprocess_ne_seq(e) for e in Y]
        bool_mask = self.get_bool_mask(Y)

        # preprocess X
        tokens = self.tokenizer.texts_to_sequences(X)
        tokens = self.padding(tokens)

        # preprocess Y
        padded_Y = self.padding(Y, dtype=object, value='O')
        targets = self.ne_stringLookup(padded_Y)

        # create dataset
        dataset = tf.data.Dataset.from_tensor_slices((tokens, targets, bool_mask))\
                    .batch(BATCH_SIZE, drop_remainder=True)

        return dataset, bool_mask, self.tokenizer, self.ne_stringLookup

In [38]:
data_creator = SeqData(X_train_raw, Y_train_raw)

train_dataset, train_bool_mask, tokenizer, ne_stringLookup = data_creator.load_dataset(X_train_raw, Y_train_raw)
dev_dataset, dev_bool_mask, _, _ = data_creator.load_dataset(X_dev_raw, Y_dev_raw)

In [42]:
example_X_batch, example_Y_batch, example_mask = next(iter(train_dataset))
print(example_X_batch.shape, example_Y_batch.shape, example_mask.shape)

(64, 930) (64, 932) (64, 932)


In [43]:
example_Y_batch

<tf.Tensor: shape=(64, 932), dtype=int64, numpy=
array([[ 2,  2, 19, ...,  0,  0,  0],
       [ 2,  2, 19, ...,  0,  0,  0],
       [ 2,  2,  0, ...,  0,  0,  0],
       ...,
       [ 2,  2,  0, ...,  0,  0,  0],
       [ 2,  2,  0, ...,  0,  0,  0],
       [ 2,  2,  0, ...,  0,  0,  0]])>

In [45]:
vocab_inp_size = len(tokenizer.word_index)+1
vocab_tar_size = len(tags)+1
max_length_input = example_X_batch.shape[1]
max_length_output = example_Y_batch.shape[1]

embedding_dim = 64
units = 64

BUFFER_SIZE = 32000
BATCH_SIZE = 64

In [46]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        self.gru_layer = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)

    def call(self, x, hidden):
        x = self.embedding(x)
        output, h = self.gru_layer(x, initial_state = hidden)
        return output, h

    def initialize_hidden_state(self):
        # initial hidden state
        return [tf.zeros((self.batch_sz, self.enc_units))] 

In [16]:
## Test Encoder Stack
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
sample_hidden = encoder.initialize_hidden_state()

# sample input
sample_output, sample_h = encoder(example_X_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h.shape))

Encoder output shape: (batch size, sequence length, units) (64, 930, 64)
Encoder h vecotr shape: (batch size, units) (64, 64)



# Test attn

In [94]:
batch_size = 4
max_time = 7
hidden_size = 32
embedding_size = 48
input_vocab_size = 128
output_vocab_size = 2

embedding_layer = tf.keras.layers.Embedding(input_vocab_size, embedding_size)
sampler = tfa.seq2seq.TrainingSampler()
output_layer = tf.keras.layers.Dense(output_vocab_size)

# mock data
input_ids = tf.random.uniform(
    [batch_size, max_time], maxval=input_vocab_size, dtype=tf.int64)

input_lengths = [5,6,2,7]
mask = [
        [True if i<valid_len else False for i in range(max_time) ] 
        for valid_len in input_lengths
]

input_tensors = embedding_layer(input_ids)

## Encoder 
encoder_lstm = tf.keras.layers.LSTM(hidden_size, return_sequences = True, return_state = True)
enc_sequences, *state = encoder_lstm(input_tensors)

In [77]:
enc_sequences.shape

TensorShape([4, 7, 32])

In [81]:
input_tensors.shape

TensorShape([4, 7, 48])

In [89]:
query.shape

TensorShape([4, 7, 32])

In [95]:
query = enc_sequences

# Attentioned RNN decoder

attention = tfa.seq2seq.LuongAttention(
        units  = 2, 
        memory = query,  # The memory to query; RNN encoder output sequence.
        memory_sequence_length = input_lengths  # valid length mask
    )

decoder_cell = tfa.seq2seq.AttentionWrapper(
    tf.keras.layers.LSTMCell(hidden_size),
    attention_mechanism = attention
)

# decoder
decoder = tfa.seq2seq.BasicDecoder(decoder_cell, sampler)


initial_state = decoder_cell.get_initial_state(input_tensors)

output, state, lengths = decoder(
    input_tensors, initial_state=initial_state,
    sequence_length = input_lengths)

logits = output.rnn_output
logits.shape


ValueError: ignored

In [98]:
input_tensors

<tf.Tensor: shape=(4, 7, 48), dtype=float32, numpy=
array([[[-0.00920322,  0.03080458, -0.04706008, ...,  0.02843005,
          0.04914875, -0.02121426],
        [ 0.018262  ,  0.04097637,  0.02961249, ..., -0.00262671,
          0.03874794, -0.01295703],
        [ 0.01946372,  0.03727869, -0.00974437, ...,  0.00460287,
         -0.02529764,  0.02750375],
        ...,
        [-0.04680122, -0.01320962, -0.00540948, ...,  0.00557275,
         -0.02082304, -0.0431443 ],
        [-0.01912578,  0.03242162,  0.00185368, ...,  0.01762482,
          0.00276135,  0.01354733],
        [-0.02686486, -0.03371592, -0.03499549, ..., -0.02348628,
          0.00045349, -0.00762911]],

       [[ 0.00581073,  0.03883186, -0.01562633, ...,  0.0219874 ,
          0.02373847, -0.03124186],
        [ 0.01369936,  0.00148897,  0.04216638, ...,  0.00099827,
         -0.02962055,  0.02090689],
        [-0.01728929, -0.04492406, -0.03196266, ..., -0.01838011,
          0.03195058, -0.02541676],
        ...,
  

In [97]:
enc_sequences

<tf.Tensor: shape=(4, 7, 32), dtype=float32, numpy=
array([[[-9.57207987e-04,  9.51110851e-03,  8.91561620e-03,
          3.22170602e-03,  1.56557432e-03, -2.11643800e-03,
         -1.78371993e-04, -7.77929462e-03,  3.15717026e-03,
          8.51548510e-04, -1.25742611e-03, -2.43971660e-03,
         -5.65995276e-03, -6.57207472e-03, -1.55241159e-03,
         -7.27468170e-03, -1.16107017e-02, -6.42238185e-03,
          1.49193616e-03, -6.37725601e-03, -6.42789435e-03,
          1.05227418e-02, -5.14355069e-03, -1.87336598e-04,
          3.28373699e-03, -5.50639071e-03,  2.34349677e-03,
          6.81360159e-03, -9.30562324e-04, -3.50032980e-03,
          2.08958238e-03, -3.59851256e-04],
        [ 1.04960217e-03,  8.68157018e-03,  1.52512984e-02,
          1.31361675e-03,  1.64882489e-03, -1.08301621e-02,
          1.79255323e-03, -3.95470671e-03,  5.39222173e-03,
         -3.71568021e-03, -6.54590828e-03, -4.98428103e-03,
         -1.77675970e-02, -8.80812109e-03, -2.74751009e-03,
    

In [24]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_type='luong'):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.attention_type = attention_type    

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.fc = tf.keras.layers.Dense(vocab_size)

    self.attention_mechanism = tfa.seq2seq.LuongAttention(
        units  = dec_units, 
        memory = memory,  # The memory to query; RNN encoder output sequence.
        memory_sequence_length = memory_sequence_length  # valid length mask
    )

    self.decoder = tfa.seq2seq.BasicDecoder(
        cell    = tfa.seq2seq.AttentionWrapper(
            cell                 = tf.keras.layers.GRUCell(self.dec_units),
            attention_mechanism  = self.attention_mechanism,
            attention_layer_size = self.dec_units
            ),
        sampler = tfa.seq2seq.sampler.TrainingSampler(),
        output_layer=self.fc
    )


    ## Decoder RNN 
    self.decoder_rnn_cell = 
   
    # Sampler
    self.sampler = 

    # Create attention mechanism with memory = None
    self.attention_mechanism = self.build_attention_mechanism(self.dec_units, None, self.batch_sz*[max_length_input], self.attention_type)

    ### Wrap Attention mechanism with the decoder RNN
    self.rnn_cell = self.build_rnn_cell(batch_sz)

    ### Define the decoder with respect to fundamental rnn cell
    self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)

    
  ''' def build_rnn_cell(self, batch_sz):
    rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell, 
                                  self.attention_mechanism, attention_layer_size=self.dec_units)
    return rnn_cell '''

  def build_attention_mechanism(self, dec_units, memory, memory_sequence_length, attention_type='luong'):
    # ------------- #
    # typ: Which sort of attention (Bahdanau, Luong)
    # dec_units: final dimension of attention outputs 
    # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
    # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)
    return 

  def build_initial_state(self, batch_sz, encoder_state, Dtype):
    decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
    return decoder_initial_state


  def call(self, inputs, initial_state):
    x = self.embedding(inputs)
    outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[max_length_output])
    return outputs


In [25]:
# Test decoder stack

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 'luong')
sample_x = tf.random.uniform((BATCH_SIZE, max_length_output))
decoder.attention_mechanism.setup_memory(sample_output)
initial_state = decoder.build_initial_state(BATCH_SIZE, sample_h, tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)

print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)


Decoder Outputs Shape:  (64, 930, 21)
